In [1]:
import pandas as pd
import pickle
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler

In [2]:
data_2020 = 'https://s3.amazonaws.com/parkerhiggins-nba-draft-bucket/19-20_MBB_StatsAndDraft.csv'
raw_2020_data = pd.read_csv(data_2020)

In [3]:
conf_data = pd.read_csv('https://s3.amazonaws.com/parkerhiggins-nba-draft-bucket/NCAA_conference_data_cleaned.csv')

In [4]:
conf_data.head()

,TEAM,CONF
0,Michigan,B10
1,Baylor,B12
2,Illinois,B10
3,Gonzaga,WCC
4,Iowa,B10


In [5]:
raw_2020_data.head()

,#,player,team,gp,mpg,fgm,fga,FG%,3PM,3PA,...,pf,orb,drb,rpg,apg,spg,bpg,ppg,Pk,Player1
0,47,Anthony Edwards,UGA,32,33.0,6.3,15.8,0.402,2.2,7.7,...,2.2,0.8,4.5,5.2,2.8,1.3,0.6,19.1,1.0,Anthony Edwards
1,1085,Patrick Williams,FSU,29,22.5,3.3,7.1,0.459,0.6,1.7,...,1.6,1.3,2.7,4.0,1.0,1.0,1.0,9.2,4.0,Patrick Williams
2,487,Isaac Okoro,AU,28,31.5,4.5,8.8,0.510,0.7,2.5,...,2.7,1.9,2.5,4.4,2.0,0.9,0.9,12.8,5.0,Isaac Okoro
3,164,Onyeka Okongwu,USC,28,30.6,6.2,10.1,0.616,0.0,0.1,...,2.7,3.3,5.4,8.6,1.1,1.2,2.7,16.2,6.0,Onyeka Okongwu
4,27,Obi Toppin,DAY,31,31.6,7.9,12.5,0.633,1.0,2.6,...,1.6,1.2,6.4,7.5,2.2,1.0,1.2,20.0,8.0,Obi Toppin


In [6]:
numerical_cols = raw_2020_data.dtypes[raw_2020_data.dtypes != 'object'].index.tolist()

numerical_df = raw_2020_data[numerical_cols]
numerical_df.index = raw_2020_data['#']

numerical_df['draft_status'] = numerical_df['Pk'].apply(lambda x: True if x <= 60 else False)

# selected_features = ['ppg','mpg','rpg','apg','spg','gp','tov','draft_status']
selected_features = ['ppg','rpg','apg','spg','tov','FG%','3P%','FT%','draft_status']
data_df = numerical_df[selected_features]

# X = numerical_df.drop(columns=['Pk','#','pf','draft_status'])
X = data_df.drop(columns=['draft_status'])
y = data_df['draft_status']

# scaler = StandardScaler()
# X = scaler.fit_transform(X)

ipykernel_launcher:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [7]:
len(selected_features)-1

8

In [29]:
model_filename = './svm_2007_2019.sav'
loaded_model = pickle.load(open(model_filename, 'rb'))

In [30]:
predictions = loaded_model.predict(X)
results = loaded_model.score(X,y)
print(results)

0.7405


In [31]:
draft_prob = loaded_model.predict_proba(X)

In [32]:
draft_prob_true = []
for prob in draft_prob:
    draft_prob_true.append(prob[1])

In [33]:
numerical_df['draft_prob_true'] = draft_prob_true
numerical_df['predictions'] = predictions

In [34]:
# numerical_df[numerical_df['Pk'] <= 60].sort_values(by='draft_prob_true',ascending=False)

In [35]:
# numerical_df.sort_values(by=['draft_prob_true'],ascending=False).head(2S)

In [36]:
acc_score = accuracy_score(y,predictions)
matrix = confusion_matrix(y,predictions)
results = pd.DataFrame({"Prediction": predictions, "Actual": y}).reset_index(drop=True)
report = classification_report(y,predictions,zero_division=True)

print(acc_score,'\n')
print(matrix,'\n')
print(report,'\n')

# numerical_df[['draft_status','draft_prob_true']].sort_values(by='draft_prob_true',ascending=False).head(39).value_counts().groupby('draft_status').count()
numerical_df[['draft_prob_true','draft_status']].sort_values(by='draft_prob_true',ascending=False).head(39).value_counts().groupby('draft_status').sum()

0.7405 

[[1447  514]
 [   5   34]] 

              precision    recall  f1-score   support

       False       1.00      0.74      0.85      1961
        True       0.06      0.87      0.12        39

    accuracy                           0.74      2000
   macro avg       0.53      0.80      0.48      2000
weighted avg       0.98      0.74      0.83      2000
 



draft_status
False    30
True      9
dtype: int64

In [37]:
# numerical_df[['draft_status','draft_prob_true']].sort_values(by='draft_prob_true',ascending=False).head(39).value_counts().groupby('draft_status').count()
# numerical_df[['draft_status','draft_prob_true']].groupby(by='draft_status',axis=1,as_index=False).count()

In [38]:
raw_2020_data['predictions'] = predictions
raw_2020_data['draft_prob'] = draft_prob_true

In [39]:
raw_2020_data.sort_values(by='draft_prob',ascending=False).head(60)

,#,player,team,gp,mpg,fgm,fga,FG%,3PM,3PA,...,drb,rpg,apg,spg,bpg,ppg,Pk,Player1,predictions,draft_prob
1019,5,Luka Garza,UI,31,32.0,9.3,17.1,0.542,1.3,3.5,...,6.3,9.8,1.2,0.8,1.8,23.9,NaN,NaN,True,0.999998
121,60,Shamarkus Kennedy,MSU,31,31.8,7.3,10.7,0.679,0.0,0.1,...,7.5,10.9,1.5,0.7,2.6,18.6,NaN,NaN,True,0.999997
344,13,Terry Taylor,APSU,33,36.6,8.6,15.7,0.550,1.2,3.9,...,7.0,11.0,1.4,1.3,1.3,21.8,NaN,NaN,True,0.999995
4,27,Obi Toppin,DAY,31,31.6,7.9,12.5,0.633,1.0,2.6,...,6.4,7.5,2.2,1.0,1.2,20.0,8.0,Obi Toppin,True,0.999981
21,26,Daniel Oturu,U of M,31,34.0,7.7,13.6,0.563,0.6,1.7,...,7.5,11.3,1.1,0.5,2.5,20.1,33.0,Daniel Oturu,True,0.996855
390,19,Nathan Knight,W&M,32,29.6,7.1,13.5,0.524,0.9,3.0,...,7.8,10.5,1.8,0.8,1.5,20.7,NaN,NaN,True,0.996833
384,168,John Mooney,ND,31,32.7,6.4,14.0,0.460,1.0,3.4,...,9.0,12.7,1.7,1.3,0.7,16.2,NaN,NaN,True,0.995945
1169,110,A.J. Brodeur,PENN,27,34.7,6.9,13.7,0.504,0.6,2.3,...,6.7,8.9,5.2,1.1,1.8,17.3,NaN,NaN,True,0.994650
1547,61,Ahsan Asadullah,LIP,31,29.9,8.1,15.6,0.520,0.1,0.5,...,7.4,10.1,3.9,1.2,1.2,18.6,NaN,NaN,True,0.994486
1368,28,Carlik Jones,RU,32,33.1,6.8,14.0,0.488,1.5,3.6,...,4.5,5.1,5.5,1.4,0.2,20.0,NaN,NaN,True,0.994315
